# Molecular volume

Arguably, the best overlap between two ligands is the one that maximises their overlapping volume. The union of atomic volumes can be calculated analytically with geometry, and a famous paper by Andrew Grant ([doi](https://doi.org/10.1002/(SICI)1096-987X(19961115)17:14<1653::AID-JCC7>3.0.CO;2-K)) showed how to do it with a Gaussian approximation of the atoms. This is nice in that the volume overlap has a gradient, and the 'fuzziness' kind of represents an electron cloud. 

But that's the union of atomic volumes. The volume of a molecule also includes the excluded spaces between atoms, and a protein shouldn't care whether the actual atoms overlap or not. 

Instead, this notebook shows how to calculate SES's and their volumes. The SES is approximated with a mesh, and there is a handy and fast algorithm for calculating the volume of a watertight mesh ([doi](https://doi.org/10.1109/ICIP.2001.958278)).

To make life simple, we start in the 2d case.


In [1]:
from rdkit.Chem.AllChem import ComputeMolVolume
ComputeMolVolume(mol.mol, 0)

NameError: name 'mol' is not defined

In [ ]:
def estimate_volume(pos, n):
    x1, y1,z1 = np.min(spheres,0)-2.5
    x2, y2,z2 = np.max(spheres,0)+2.5
    x = np.random.uniform(x1, x2, n)
    y = np.random.uniform(y1, y2, n)
    z = np.random.uniform(z1, z2, n)
    s = np.vstack([x,y,z]).T
    dis = (cdist(s, spheres) - radii).min(1)
    i = (dis<0).sum()
    return (i / n) * ((x2-x1) * (y2-y1) * (z2-z1))

    
    
spheres = mol.mol.GetConformer(0).GetPositions()

sizes = [100, 250, 500, 1000, 1500, 2000, 2500, 3000, 3500, 5000, 10_000, 12500, 15000, 20000]
results = {}
repeats = 5
for size in sizes:
    results[size] = []
    for i in range(repeats):
        vol = estimate_volume(spheres, size)
        results[size].append(vol)

In [ ]:
for size in sizes:
    plt.scatter(np.ones(repeats)*size, results[size])
plt.xscale('log')


In [ ]:
import meshplot as mp

In [ ]:
%%time
import igl 


n=500
x1, y1,z1 = np.min(vert,0)-1.5
x2, y2,z2 = np.max(vert,0)+1.5
x = np.random.uniform(x1, x2, n)
y = np.random.uniform(y1, y2, n)
z = np.random.uniform(z1, z2, n)
s = np.vstack([x,y,z]).T
#dis = (cdist(s, spheres) - radii).min(1)
#i = (dis<0).sum()
#return (i / n) * ((x2-x1) * (y2-y1) * (z2-z1))




sd, i, c = igl.signed_distance(s, vert, faces)
inside = (sd<0).sum()
print((inside / n) * ((x2-x1) * (y2-y1) * (z2-z1)))

In [ ]:
def points_to_barycentric(triangles,
                          points,
                          method='cramer'):
    """
    Find the barycentric coordinates of points relative to triangles.
    The Cramer's rule solution implements:
        http://blackpawn.com/texts/pointinpoly
    The cross product solution implements:
        https://www.cs.ubc.ca/~heidrich/Papers/JGT.05.pdf
    Parameters
    -----------
    triangles : (n, 3, 3) float
      Triangles vertices in space
    points : (n, 3) float
      Point in space associated with a triangle
    method :  str
      Which method to compute the barycentric coordinates with:
        - 'cross': uses a method using cross products, roughly 2x slower but
                  different numerical robustness properties
        - anything else: uses a cramer's rule solution
    Returns
    -----------
    barycentric : (n, 3) float
      Barycentric coordinates of each point
    """

    def method_cross():
        n = np.cross(edge_vectors[:, 0], edge_vectors[:, 1])
        denominator = diagonal_dot(n, n)

        barycentric = np.zeros((len(triangles), 3), dtype=np.float64)
        barycentric[:, 2] = diagonal_dot(
            np.cross(edge_vectors[:, 0], w), n) / denominator
        barycentric[:, 1] = diagonal_dot(
            np.cross(w, edge_vectors[:, 1]), n) / denominator
        barycentric[:, 0] = 1 - barycentric[:, 1] - barycentric[:, 2]
        return barycentric

    def method_cramer():
        dot00 = diagonal_dot(edge_vectors[:, 0], edge_vectors[:, 0])
        dot01 = diagonal_dot(edge_vectors[:, 0], edge_vectors[:, 1])
        dot02 = diagonal_dot(edge_vectors[:, 0], w)
        dot11 = diagonal_dot(edge_vectors[:, 1], edge_vectors[:, 1])
        dot12 = diagonal_dot(edge_vectors[:, 1], w)

        inverse_denominator = 1.0 / (dot00 * dot11 - dot01 * dot01)

        barycentric = np.zeros((len(triangles), 3), dtype=np.float64)
        barycentric[:, 2] = (dot00 * dot12 - dot01 *
                             dot02) * inverse_denominator
        barycentric[:, 1] = (dot11 * dot02 - dot01 *
                             dot12) * inverse_denominator
        barycentric[:, 0] = 1 - barycentric[:, 1] - barycentric[:, 2]
        return barycentric

    # establish that input triangles and points are sane
    triangles = np.asanyarray(triangles, dtype=np.float64)
    points = np.asanyarray(points, dtype=np.float64)
#     if not util.is_shape(triangles, (-1, 3, 3)):
#         raise ValueError('triangles shape incorrect')
#     if not util.is_shape(points, (len(triangles), 3)):
#         raise ValueError('triangles and points must correspond')

    edge_vectors = triangles[:, 1:] - triangles[:, :1]
    w = points - triangles[:, 0].reshape((-1, 3))

    if method == 'cross':
        return method_cross()
    return method_cramer()

def diagonal_dot(a, b):
    """From trimesh repo"""
    a = np.asanyarray(a)
    # 3x faster than (a * b).sum(axis=1)
    # avoiding np.ones saves 5-10% sometimes
    return np.dot(a * b, [1.0] * a.shape[1])

In [ ]:
points_to_barycentric(vert[faces], np.random.random([924,3]))

In [ ]:
# // Compute barycentric coordinates (u, v, w) for
# // point p with respect to triangle (a, b, c)
# void Barycentric(Point p, Point a, Point b, Point c, float &u, float &v, float &w)
# {
#     Vector v0 = b - a, v1 = c - a, v2 = p - a;
#     float d00 = Dot(v0, v0);
#     float d01 = Dot(v0, v1);
#     float d11 = Dot(v1, v1);
#     float d20 = Dot(v2, v0);
#     float d21 = Dot(v2, v1);
#     float denom = d00 * d11 - d01 * d01;
#     v = (d11 * d20 - d01 * d21) / denom;
#     w = (d00 * d21 - d01 * d20) / denom;
#     u = 1.0f - v - w;
# }
#https://gamedev.stackexchange.com/questions/23743/whats-the-most-efficient-way-to-find-barycentric-coordinates

def bary(p, t):
    a,b,c = t

    v0 = b - a; v1 = c - a; v2 = p - c;
    d00 = np.dot(v0, v0);
    d01 = np.dot(v0, v1);
    d11 = np.dot(v1, v1);
    d20 = np.dot(v2, v0);
    d21 = np.dot(v2, v1);
    denom = d00 * d11 - d01 * d01;

    v = (d11 * d20 - d01 * d21) / denom;

    w = (d00 * d21 - d01 * d20) / denom;
    u = 1.0 -v -w 
    return v, w, u 

    

In [ ]:
from jax import vmap, jit
import jax.numpy as jnp

@jit
def signed_vol_of_triangle(p1, p2, p3):
    v321 = p3[0]*p2[1]*p1[2]
    v231 = p2[0]*p3[1]*p1[2]
    v312 = p3[0]*p1[1]*p2[2]
    v132 = p1[0]*p3[1]*p2[2]
    v213 = p2[0]*p1[1]*p3[2]
    v123 = p1[0]*p2[1]*p3[2]
    return (1 / 6)*(-v321 + v231 + v312 - v132 - v213 + v123)


def make_vol(pts):
    return signed_vol_of_triangle(pts[0], pts[1], pts[2])

vmake_vol = vmap(make_vol)

In [ ]:
tris = vert[faces]



In [ ]:
%%time
vmake_vol(np.vstack([tris, -1*(tris+2)])).sum() 

In [ ]:
import open3d as o3d
chimera_mesh = o3d.io.read_triangle_mesh('chimera.stl')

chimera_vertices = np.array(chimera_mesh.vertices)
chimera_faces = np.array(chimera_mesh.triangles)

o3d.visualization.draw_geometries([chimera_mesh])

In [ ]:
vmake_vol(chimera_vertices[chimera_faces]).sum()


In [ ]:
vmake_vol((vert)[faces]).sum()

In [ ]:
import igl 
lv, lf = igl.loop(vert, faces.astype(int))
v = lv.copy()
f = lf.copy()